<a href="https://colab.research.google.com/github/K-Hebel/WiDS_Chi/blob/master/Model_WiDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Libraries

In [4]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from xgboost import XGBClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics 

#Load DataFile 'final_sample.csv' - A 50,000 record sample of orginal training dataset (Cleaned Based on EDA comments -df_A)

In [27]:
df_A=pd.read_csv('/content/drive/My Drive/final_sample.csv')

In [28]:
df_A.describe()

,Unnamed: 0,age,bmi,weight,arf_apache,bun_apache,creatinine_apache,glucose_apache,d1_diasbp_invasive_min,d1_bun_max,d1_bun_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hemaglobin_max,d1_potassium_max,h1_bun_max,h1_bun_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_potassium_max,h1_potassium_min,aids,cirrhosis,hepatic_failure,solid_tumor_with_metastasis,diabetes_mellitus
count,50000.00000,48413.000000,48297.000000,48671.000000,50000.000000,40759.000000,40961.000000,45802.000000,13461.000000,45143.000000,45143.000000,45312.000000,45312.000000,47683.000000,47683.000000,43867.000000,45627.000000,10142.000000,10142.000000,10219.000000,10219.000000,23530.000000,23530.000000,11878.000000,11878.000000,50000.000000,50000.000000,50000.00000,50000.00000,50000.000000
mean,64517.25396,0.706565,0.287905,0.325286,0.041300,0.193007,0.120662,0.256355,0.491915,0.193565,0.202304,0.119614,0.122323,0.232773,0.333383,0.436856,0.356713,0.179845,0.179676,0.116601,0.116768,0.202849,0.215413,0.367910,0.367195,0.000780,0.017040,0.01368,0.01968,0.500000
std,37304.10042,0.179790,0.162890,0.175116,0.198985,0.175740,0.152694,0.185362,0.149649,0.175884,0.177891,0.153284,0.150881,0.184414,0.167973,0.204684,0.163933,0.170784,0.170673,0.149090,0.149336,0.170662,0.162622,0.166125,0.168320,0.027918,0.129422,0.11616,0.13890,0.500005
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000
25%,31991.25000,0.606742,0.177694,0.200814,0.000000,0.073171,0.040441,0.110774,0.404762,0.073770,0.081751,0.041783,0.045653,0.102230,0.227451,0.278846,0.238095,0.068702,0.068702,0.041688,0.041811,0.092761,0.111465,0.255319,0.260870,0.000000,0.000000,0.00000,0.00000,0.000000
50%,64325.50000,0.730337,0.258489,0.298507,0.000000,0.130081,0.068934,0.219761,0.488095,0.131148,0.145336,0.067781,0.072630,0.182156,0.301961,0.432692,0.333333,0.122137,0.122137,0.066524,0.066720,0.150933,0.167197,0.340426,0.347826,0.000000,0.000000,0.00000,0.00000,0.500000
75%,96233.75000,0.842697,0.363143,0.416554,0.000000,0.252033,0.128676,0.343041,0.571429,0.245902,0.263421,0.126277,0.127621,0.306691,0.403922,0.576923,0.428571,0.229008,0.229008,0.121516,0.121875,0.253127,0.265924,0.446809,0.456522,0.000000,0.000000,0.00000,0.00000,1.000000
max,130155.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000


##Proportion of Missing Data in df_A by Feature Column and Count of Available data by Feature Column

In [29]:
missing_data= pd.DataFrame(df_A.isna().sum()/df_A['diabetes_mellitus'].count())
missing_data['Available']= df_A.notna().sum()
missing_data.sort_values(0,ascending=False)

,0,Available
h1_bun_max,0.79716,10142
h1_bun_min,0.79716,10142
h1_creatinine_max,0.79562,10219
h1_creatinine_min,0.79562,10219
h1_potassium_min,0.76244,11878
h1_potassium_max,0.76244,11878
d1_diasbp_invasive_min,0.73078,13461
h1_glucose_max,0.52940,23530
h1_glucose_min,0.52940,23530
bun_apache,0.18482,40759


##Set Features and Target (df_A) and Train_Test_split

In [30]:
y_a=df_A['diabetes_mellitus']
X_a=df_A.drop(['diabetes_mellitus'], axis=1)

X_a_train, X_a_test,y_a_train, y_a_test =train_test_split(X_a,y_a, random_state=40)

In [31]:
y_a.shape, X_a.shape

((50000,), (50000, 29))

In [32]:
y_a_train.shape, X_a_train.shape

((37500,), (37500, 29))

#Load Original DataFile fo 130,157 records (Clean Based on Prop of Available Data - df_B)

In [6]:
#@title
! pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khebel","key":"11c0f35aedbce6808a3f45f82709accf"}'}

In [7]:
#@title
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'widsdatathon2021'
! unzip TrainingWiDS2021.csv.zip -d train
! unzip UnlabeledWiDS2021.csv.zip -d train

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/90.0k [00:00<?, ?B/s]
100% 90.0k/90.0k [00:00<00:00, 34.0MB/s]
 42% 9.00M/21.5M [00:00<00:00, 37.3MB/s]
100% 21.5M/21.5M [00:00<00:00, 61.4MB/s]
  0% 0.00/1.60M [00:00<?, ?B/s]
100% 1.60M/1.60M [00:00<00:00, 109MB/s]
  0% 0.00/59.0 [00:00<?, ?B/s]
100% 59.0/59.0 [00:00<00:00, 58.3kB/s]
  0% 0.00/28.2k [00:00<?, ?B/s]
100% 28.2k/28.2k [00:00<00:00, 28.7MB/s]
Archive:  TrainingWiDS2021.csv.zip
  inflating: train/TrainingWiDS2021.csv  
Archive:  UnlabeledWiDS2021.csv.zip
  inflating: train/UnlabeledWiDS2021.csv  


In [53]:
n#@title
df_B=pd.read_csv("train/TrainingWiDS2021.csv")



##Proportion of Missing Data in df_B by Feature Column and Count of Available data by Feature Column

In [13]:
missing_data_B= pd.DataFrame(df_B.isna().sum()/df_B['diabetes_mellitus'].count())
missing_data_B['Available']= df_B.notna().sum()
missing_data_B.sort_values(0,ascending=False)

,0,Available
h1_bilirubin_max,0.920896,10296
h1_bilirubin_min,0.920896,10296
h1_albumin_max,0.914319,11152
h1_albumin_min,0.914319,11152
h1_lactate_min,0.910185,11690
...,...,...
icu_stay_type,0.000000,130157
icu_id,0.000000,130157
elective_surgery,0.000000,130157
hospital_id,0.000000,130157


##Drop Columns with missing data >=70% from df_B

In [14]:
col_drop=missing_data_B[missing_data_B[0]>= 0.70].sort_values(by=0, ascending=False)
df_B= df_B.drop(list(col_drop.index), axis=1) #[0].sort_values
df_B.shape

(130157, 126)

##Create Stratified Sample of df_B dataset to balance target variable value counts for sample size of 56,302 (included all positive diabetes_mellitus from original dataset)

In [15]:
df_B['diabetes_mellitus'].value_counts()

0    102006
1     28151
Name: diabetes_mellitus, dtype: int64

In [16]:
sample_df_B=df_B.groupby('diabetes_mellitus', group_keys=False).apply(lambda x: x.sample(28151))
sample_df_B.describe()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,...,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_glucose_max,h1_glucose_min,h1_inr_max,h1_inr_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,56302.000000,56302.000000,56302.000000,54458.000000,54328.000000,56302.000000,55476.000000,56302.000000,56302.000000,56302.0,54743.000000,22778.000000,55671.000000,56015.000000,56302.000000,56302.000000,20752.000000,45867.000000,46117.000000,55498.000000,55498.000000,56068.000000,55498.000000,51475.000000,56196.000000,44855.000000,56302.000000,56159.000000,55964.000000,46244.000000,53451.000000,28088.000000,56302.000000,43729.000000,56203.000000,56203.000000,55580.000000,55580.000000,56205.000000,56205.000000,...,49590.000000,49590.000000,51011.000000,51011.000000,53599.000000,53599.00000,47940.000000,47940.000000,49366.000000,49366.000000,49602.000000,49602.000000,21005.000000,21005.000000,48378.000000,48378.000000,51339.000000,51339.000000,50974.000000,50974.000000,48852.000000,48852.000000,26363.000000,26363.000000,21005.000000,21005.000000,20245.000000,20245.000000,20104.000000,20104.000000,20465.000000,20465.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000
mean,64551.810131,212912.001279,106.303151,62.956186,30.066446,0.189514,169.580691,658.520106,0.878027,0.0,86.450158,2.873725,185.679456,563.244361,0.204327,0.039714,1.145418,27.785774,1.606552,3.505604,5.505586,0.010166,4.053624,182.077397,99.441757,32.713524,0.156762,87.444772,25.035484,137.810840,36.415964,1801.366680,0.335299,12.204489,88.352116,49.400584,88.520961,49.463530,102.914385,70.644373,...,8.404209,8.180381,1.620572,1.473878,197.787086,117.98013,24.426252,23.035918,11.342128,10.799155,34.266655,32.690454,1.589954,1.476145,207.238642,196.386711,4.296942,3.944436,139.107174,137.540432,12.553873,11.340639,188.066391,177.240033,1.589954,1.476145,45.294231,38.563100,7.386328,7.321183,161.979281,101.665506,0.000781,0.017086,0.014369,0.024653,0.007424,0.003996,0.019307,0.500000
std,37430.793248,38121.071346,63.829115,15.994651,8.662105,0.391920,10.817723,303.566119,2.573847,0.0,25.914414,0.677071,86.407297,462.033281,0.403212,0.195289,2.200029,21.622117,1.642050,0.923149,1.248765,0.100315,1.524534,103.536305,30.387299,6.715500,0.363579,42.236010,15.016875,5.302905,0.854454,1453.696187,0.472099,6.867050,20.514825,13.315746,20.503206,13.344687,21.695048,17.033795,...,0.734835,0.784170,1.628564,1.435552,99.035518,42.69092,4.447521,5.251120,2.125252,2.298428,6.126449,6.700914,0.964036,0.751640,89.175272,87.589629,0.688323,0.593603,4.832746,4.920581,6.729382,5.879717,108.425017,101.889647,0.964036,0.751640,14.617342,11.121994,0.086257,0.111102,104.654631,60.215500,0.027945,0.129595,0.119007,0.155066,0.085844,0.063090,0.137602,0.500004
min,1.000000,147000.000000,1.000000,0.000000,14.844926,0.000000,137.200000,82.000000,-0.219444,0.0,38.600000,1.200000,101.000000,0.010000,0.000000,0.000000,0.100000,4.0

In [17]:
sample_df_B['diabetes_mellitus'].value_counts()

1    28151
0    28151
Name: diabetes_mellitus, dtype: int64

##Drop Features Not Associated with Patient Vitals

In [18]:
drop_cols2 =['Unnamed: 0',
 'encounter_id',
 'hospital_id',
 'elective_surgery',
 'hospital_admit_source',
 'icu_admit_source',
 'icu_id',
 'icu_stay_type',
 'icu_type',
 'pre_icu_los_days',
 'readmission_status',
 ]

sample_df_B2=sample_df_B.drop(drop_cols2, axis=1)
sample_df_B2.shape

(56302, 115)

##Encode Categorical Data in sample_df_B2 (Sample set of df_B)

In [19]:
sample_df_B2.select_dtypes(include=['object']).columns

Index(['ethnicity', 'gender'], dtype='object')

In [20]:
demo_cat=pd.get_dummies(sample_df_B2[['gender','ethnicity']])
sample_df_B3 = sample_df_B2.join(demo_cat)
sample_df_B3=sample_df_B3.drop(['gender','ethnicity'], axis=1)
sample_df_B3.shape

(56302, 121)

##Normalize All Numerical Data in sample_df  using a max min scaler
lambda x: (x - x.min()) / (x.max() - x.min()) (sample_df_B4)

In [21]:
sample_df_B4 = sample_df_B3.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
sample_df_B4.describe()

,age,bmi,height,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,...,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_glucose_max,h1_glucose_min,h1_inr_max,h1_inr_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus,gender_F,gender_M,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown
count,54458.000000,54328.000000,55476.000000,54743.000000,22778.000000,55671.000000,56015.000000,56302.000000,56302.000000,20752.000000,45867.000000,46117.000000,55498.000000,55498.000000,56068.000000,55498.000000,51475.000000,56196.000000,44855.000000,56302.000000,56159.000000,55964.000000,46244.000000,53451.000000,28088.000000,56302.000000,43729.000000,56203.000000,56203.000000,55580.000000,55580.000000,56205.000000,56205.000000,56191.000000,56191.000000,55309.000000,55309.000000,56022.000000,56022.000000,56108.000000,...,49366.000000,49366.000000,49602.000000,49602.000000,21005.000000,21005.000000,48378.000000,48378.000000,51339.000000,51339.000000,50974.000000,50974.000000,48852.000000,48852.000000,26363.000000,26363.000000,21005.000000,21005.000000,20245.000000,20245.000000,20104.000000,20104.000000,20465.000000,20465.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000,56302.000000
mean,0.707373,0.287361,0.554559,0.324628,0.492272,0.409079,0.255895,0.204327,0.039714,0.017395,0.193380,0.120087,0.835201,0.901117,0.010166,0.763406,0.255632,0.469201,0.469134,0.156762,0.296530,0.375634,0.507581,0.567890,0.206658,0.335299,0.251770,0.355900,0.472735,0.357319,0.473552,0.377432,0.403682,0.358907,0.468311,0.367655,0.468793,0.189332,0.125971,0.992935,...,0.436743,0.482382,0.445873,0.489394,0.100635,0.110225,0.323008,0.329999,0.356415,0.454246,0.460205,0.570568,0.252983,0.261029,0.202920,0.215350,0.100635,0.110225,0.290434,0.333227,0.586948,0.645618,0.245045,0.175022,0.000781,0.017086,0.014369,0.024653,0.007424,0.003996,0.019307,0.500000,0.457142,0.542396,0.115129,0.017726,0.756492,0.040158,0.008295,0.050869
std,0.179715,0.163528,0.185267,0.175810,0.199139,0.417427,0.209916,0.403212,0.195289,0.036606,0.175790,0.150924,0.307716,0.249753,0.100315,0.381134,0.184985,0.205320,0.190781,0.363579,0.263975,0.268158,0.129339,0.112428,0.166772,0.472099,0.152941,0.172393,0.172932,0.172296,0.173308,0.182311,0.097336,0.171373,0.171804,0.174731,0.172012,0.135679,0.050703,0.017290,...,0.204351,0.201617,0.196992,0.197667,0.140612,0.143799,0.159812,0.162534,0.163886,0.174589,0.138078,0.136683,0.149942,0.147000,0.170468,0.162245,0.140612,0.143799,0.157855,0.156621,0.152474,0.166355,0.208531,0.143066,0.027945,0.129595,0.119007,0.155066,0.085844,0.063090,0.137602,0.500004,0.498164,0.498204,0.319181,0.131954,0.429203,0.196332,0.090697,0.219731
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

##Set Features and Target (sample of df_B) & Training, Test Split

In [22]:
y_b=sample_df_B4['diabetes_mellitus']
X_b=sample_df_B4.drop(['diabetes_mellitus'], axis=1)

X_b_train, X_b_test, y_b_train,y_b_test =train_test_split(X_b,y_b, random_state=40)

In [23]:
y_b_train.shape, X_b_train.shape

((42226,), (42226, 120))

In [24]:
y_b_test.shape, X_b_test.shape

((14076,), (14076, 120))

#Create Training and Predictive Models For df_A (Sample set of 50,000)

In [25]:
model_1_A= XGBClassifier() #used to manage datasets with missing values
model_2_A= Pipeline(([('imputer', KNNImputer()), ('clf', RandomForestClassifier())]))
model_3_A = Pipeline(([('imputer', KNNImputer()), ('clf', LogisticRegression())]))


## XGBClassifier Model 1 - df_A

In [33]:
model_1_A.fit(X_a_train,y_a_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [34]:
model_1_A.score(X_a_test,y_a_test)

0.76552

##RandomForestClassifier Model 2- df_A

In [35]:
model_2_A.fit(X_a_train,y_a_train)

Pipeline(memory=None,
         steps=[('imputer',
                 KNNImputer(add_indicator=False, copy=True,
                            metric='nan_euclidean', missing_values=nan,
                            n_neighbors=5, weights='uniform')),
                ('clf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=No

In [36]:
model_2_A.score(X_a_test,y_a_test)

0.73952

#Create Training and Predictive Models For df_B (Sample set of 56,302)

In [41]:
model_1_B= XGBClassifier() #used to manage datasets with missing values
model_2_B= Pipeline(([('imputer', KNNImputer()), ('clf', RandomForestClassifier())]))


## XGBClassifier Model 1 - df_B

In [42]:
model_1_B.fit(X_b_train,y_b_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [43]:
model_1_B.score(X_b_test,y_b_test)

0.7681159420289855

##RandomForestRegressor 2 - df_B

In [44]:
model_2_B.fit(X_b_train,y_b_train)

Pipeline(memory=None,
         steps=[('imputer',
                 KNNImputer(add_indicator=False, copy=True,
                            metric='nan_euclidean', missing_values=nan,
                            n_neighbors=5, weights='uniform')),
                ('clf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=No

In [50]:
model_2_B.score(X_b_test,y_b_test)

0.7505683432793407

###Results reflect that XGBClassifier and RandomForest Classifier are the strongest predictive models

##Parameter Tweeking

In [ ]:
##Parameter Tweeking

#Using Unlabled Data

In [8]:
unlabeled_data=pd.read_csv('train/UnlabeledWiDS2021.csv')

In [57]:
unlabeled_data.describe()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,10234.000000,10234.000000,10234.000000,10234.000000,9219.000000,10234.000000,9933.000000,10234.000000,10234.000000,10234.0,9326.000000,4179.000000,10064.000000,10126.000000,10234.000000,10234.000000,3924.000000,8173.000000,8191.000000,2599.000000,10029.000000,10029.000000,10173.000000,10029.000000,9024.000000,10204.000000,8120.000000,10234.000000,10182.000000,2599.000000,2599.000000,2599.000000,2599.000000,10170.000000,8241.000000,9563.000000,5044.000000,10234.000000,7858.000000,2909.000000,...,1896.000000,2189.000000,2189.000000,2195.000000,2195.000000,4198.000000,4198.000000,813.000000,813.000000,2038.000000,2038.000000,2355.000000,2355.000000,2269.000000,2269.000000,1970.000000,1970.000000,3823.000000,3823.000000,3749.000000,3749.000000,3892.000000,3892.000000,3122.000000,3122.000000,1887.000000,1887.000000,1833.000000,1833.000000,1928.000000,1928.000000,1422.000000,1422.000000,10234.000000,10234.000000,10234.000000,10234.000000,10234.000000,10234.000000,10234.000000
mean,5117.500000,140498.779461,10092.391929,62.853625,29.116463,0.200117,169.276414,677.823725,0.831339,0.0,83.478729,2.865877,187.961447,570.656532,0.219660,0.019250,1.148081,25.110221,1.397522,0.608391,3.497158,5.494965,0.014155,4.062419,161.399236,98.919922,32.888448,0.169533,85.461697,41.165092,41.165092,136.852546,7.358241,24.284435,138.011285,36.433963,1943.528351,0.328611,12.017957,80.118589,...,22.512500,11.220401,11.082832,33.668074,33.144970,1.591403,1.454516,2.969648,2.906861,188.520981,188.036192,4.155992,4.107973,138.433308,138.061338,13.139518,13.102954,44.669704,37.769997,7.393215,7.326004,173.481308,103.107914,297.799340,224.415951,43.364049,42.196057,7.345563,7.331446,176.303994,156.767998,254.687589,246.255826,0.000977,0.012019,0.010064,0.022572,0.006547,0.004104,0.017686
std,2954.445661,3182.455522,57.008062,17.851661,8.342873,0.400107,10.805701,304.116531,2.411936,0.0,24.613964,0.709070,86.066103,477.423260,0.414036,0.137408,2.235654,20.464683,1.373137,0.271831,0.939067,1.267596,0.118136,1.521123,87.738124,30.846816,6.679005,0.375240,41.588964,11.808366,11.808366,91.905812,0.094708,15.157814,5.312423,0.912793,1675.662272,0.469731,6.868647,22.556306,...,4.907947,2.336452,2.399290,6.730747,6.982436,0.993214,0.729029,2.752060,2.692012,89.867998,90.111379,0.725008,0.713423,5.694783,5.589341,6.618158,6.604238,14.140818,10.581900,0.081752,0.107496,113.066388,63.764450,136.576568,119.435927,13.984545,13.710472,0.103545,0.109309,118.798786,105.623078,132.087636,130.745488,0.031245,0.108975,0.099821,0.148541,0.080651,0.063934,0.131814
min,1.000000,135000.000000,10001.000000,18.000000,14.900000,0.000000,137.000000,82.000000,-0.208333,0.0,38.100000,1.100000,101.000000,0.100000,0.000000,0.000000,0.100000,4.000000,0